In [1]:
%matplotlib widget

In [2]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCDof, BCSlice
from ibvpy.api import XDomainFEGrid
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import MATS2DElastic
from ibvpy.tmodel.viz3d_tensor_field import \
   Vis3DTensorField, Viz3DTensorField

In [3]:
B = 1
fe_grid = XDomainFEGrid(integ_factor = B,
                        coord_min=(0,0),
                        coord_max=(1,1),
                        shape=(10,10),
                        fets=FETS2D4Q())

In [4]:
mat_elastic = MATS2DElastic(E=28000, nu=0.2)

In [5]:
ts = TStepBC(
    domains = [(fe_grid, mat_elastic)]
)

In [6]:
fix_left = BCSlice(slice=fe_grid.mesh[0, 3:, 0, :], var='u',
                   dims=[0,1], value=0 )
load_right = BCSlice(slice=fe_grid.mesh[-1,:,-1,:],
                    var='u', dims=[1], value=0.1)
ts.bc = [fix_left, load_right]

In [7]:
ts.hist.vis_record = {
    'stress' : Vis3DTensorField(var='sig_ab')
}

In [8]:
sim = ts.sim
sim.tloop.verbose=True
sim.tline.step=1
sim.reset()
sim.run()

t:	 0.00(0), 
	 1.00(1), 



In [9]:
U_1 = ts.hist.U_t[-1]

In [10]:
eps_Emab = fe_grid.map_U_to_field(U_1)

In [11]:
eps_Emab[3,0]

array([[ 0.08516775,  0.03104485],
       [ 0.03104485, -0.00359675]])

In [12]:
sig_tip, _ = mat_elastic.get_corr_pred(eps_Emab[3,:],1)

In [13]:
sig_tip

array([[[2463.07839181,  724.37980758],
        [ 724.37980758,  391.90672802]],

       [[2405.75714825,  331.74294693],
        [ 331.74294693,  105.3005102 ]],

       [[1424.16499662,  217.1004598 ],
        [ 217.1004598 ,  -91.01792012]],

       [[1481.48624019,  609.73732045],
        [ 609.73732045,  195.5882977 ]]])

In [14]:
if True:
    from mayavi import mlab
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False
    f_stress = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    stress_viz = Viz3DTensorField(vis3d=ts.hist['stress'])
    stress_viz.setup()
    stress_viz.plot(ts.t_n)
    mlab.show()